In [4]:
#imports
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import patsy
#import statsmodels.api as sm
from scipy.stats import ttest_ind
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import seaborn as sns
plt.style.use('bmh')

In [5]:
park_counts = pd.read_csv('Parking_Counts.csv')
park_counts = park_counts.loc[park_counts['longitude'] != -180.0 ] #remove the outlier
park_counts.rename(columns={'pole_id':'id'}, inplace= True)
park_counts['type'] = 'parking'
del park_counts['Unnamed: 0']
park_counts 

,total_count,id,longitude,latitude,midday,am_early,am_peak,pm_peak,pm_late,type
0,10.767183,1-1004,-117.163929,32.715904,5.096620,0.298451,2.066197,3.191127,0.114789,parking
1,11.283732,1-1006,-117.163930,32.716037,5.578732,0.286972,1.905493,3.455141,0.057394,parking
2,10.927887,1-1008,-117.163931,32.716169,5.050704,0.137746,2.261338,3.432183,0.045915,parking
3,10.044014,1-1020,-117.161278,32.717890,3.776549,0.160704,1.882535,4.166831,0.057394,parking
4,12.362746,1-1310,-117.163951,32.719024,7.105423,0.573944,3.420704,1.251197,0.011479,parking
5,12.718592,1-1312,-117.163952,32.719161,7.713803,0.321408,3.524014,1.124930,0.034437,parking
6,12.282394,1-1313,-117.163770,32.719298,7.426831,0.309930,3.546972,0.975704,0.022958,parking
7,13.326972,1-1314,-117.163953,32.719453,7.644930,0.516549,3.799507,1.354507,0.011479,parking
8,11.111549,1-1315,-117.163772,32.719571,6.784014,0.149225,3.122254,1.056056,0.000000,parking
9,11.203380,1-1317,-117.163772,32.719707,6.956197,0.195141,3.351831,0.700211,0.000000,parking


In [6]:
restaurants = pd.read_csv('Restaurant_Counts.csv')
restaurants.columns = ['id', 'title', 'zip', 'am_early', 'am_peak', 'midday', 'pm_peak', 'pm_late', 'type', 'latitude', 'longitude', 'total_count']
restaurants['total_count'] = restaurants['total_count'].apply( lambda x : x * 2/3)
restaurants['am_peak'] = restaurants['am_peak'].apply( lambda x : x * 2/3)
restaurants['am_early'] = restaurants['am_early'].apply( lambda x : x * 2/3)
restaurants['midday'] = restaurants['midday'].apply( lambda x : x * 2/3)
restaurants['pm_peak'] = restaurants['pm_peak'].apply( lambda x : x * 2/3)
restaurants['pm_late'] = restaurants['pm_late'].apply( lambda x : x * 2/3)
restaurants['id'] = restaurants['id'].apply(lambda x : ('R-' + str(x) ))
restaurants = restaurants[['id','type','title', 'am_early', 'am_peak', 'midday', 'pm_peak', 'pm_late', 'total_count','latitude', 'longitude']]

restaurants

,id,type,title,am_early,am_peak,midday,pm_peak,pm_late,total_count,latitude,longitude
0,R-0,Only Food,c r e a m,2,16,13,12,3,46,32.767243,-117.096294
1,R-1,Only Food,snow cones y raspados,2,16,13,12,3,46,32.697974,-117.096250
2,R-2,Food & Drinks,jalapeno taco shop,4,18,16,23,16,79,32.748683,-117.126968
3,R-3,Only Food,tacos el campechano inc,2,16,13,12,3,46,32.697974,-117.096250
4,R-4,Only Food,up2you cafe llc,2,16,13,12,3,46,32.718370,-117.157817
5,R-5,Food & Drinks,awash ethiopian restaurant,4,18,16,23,16,79,32.677079,-117.107167
6,R-6,Only Food,gueros taco shop,2,16,13,12,3,46,32.890137,-117.150878
7,R-7,Food & Drinks,the fire spot,4,18,16,23,16,79,32.773069,-117.156144
8,R-8,Only Drinks,up2you cafe llc,18,4,2,18,20,62,33.016118,-117.075608
9,R-9,Food & Drinks,cold beers & cheeseburgers,4,18,16,23,16,79,32.708791,-117.160357


In [7]:
transit = pd.read_csv('Stop_Counts.csv')
transit.rename(columns={'stop_id':'id'}, inplace= True)
transit['type'] = 'transit_stop'
del transit['Unnamed: 0'] 
transit

,total_count,id,latitude,longitude,am_early,am_peak,midday,pm_peak,pm_late,type
0,214,75000,32.54,-117.03,48.0,36.0,40.0,70.0,20.0,transit_stop
1,214,75000,32.54,-117.03,48.0,36.0,40.0,70.0,20.0,transit_stop
2,101,75002,32.56,-117.05,21.0,18.0,20.0,32.0,10.0,transit_stop
3,98,75003,32.56,-117.05,20.0,16.0,20.0,32.0,10.0,transit_stop
4,101,75004,32.57,-117.07,21.0,18.0,20.0,32.0,10.0,transit_stop
5,101,75005,32.57,-117.07,23.0,17.0,19.0,32.0,10.0,transit_stop
6,101,75006,32.59,-117.08,20.0,19.0,20.0,32.0,10.0,transit_stop
7,95,75007,32.59,-117.08,16.0,17.0,20.0,32.0,10.0,transit_stop
8,101,75008,32.60,-117.08,20.0,19.0,20.0,31.0,11.0,transit_stop
9,99,75009,32.60,-117.08,21.0,16.0,20.0,32.0,10.0,transit_stop


In [8]:
location_counts = restaurants.append(park_counts).append(transit)
# Setting a new index
#location_counts.index = idx # new ad hoc index
# location_counts.index = range(len(location_counts)) # set with list
location_counts = location_counts.reset_index() # replace old w new
# location_counts.rename(columns={'index':'idx'}, inplace= True)
# location_counts.rename(columns={'id':'type_id'}, inplace= True)
# del location_counts['Unnamed: 0']
del location_counts['index']
location_counts

,am_early,am_peak,id,latitude,longitude,midday,pm_late,pm_peak,title,total_count,type
0,2.0,16.0,R-0,32.767243,-117.096294,13.0,3.0,12.0,c r e a m,46.0,Only Food
1,2.0,16.0,R-1,32.697974,-117.096250,13.0,3.0,12.0,snow cones y raspados,46.0,Only Food
2,4.0,18.0,R-2,32.748683,-117.126968,16.0,16.0,23.0,jalapeno taco shop,79.0,Food & Drinks
3,2.0,16.0,R-3,32.697974,-117.096250,13.0,3.0,12.0,tacos el campechano inc,46.0,Only Food
4,2.0,16.0,R-4,32.718370,-117.157817,13.0,3.0,12.0,up2you cafe llc,46.0,Only Food
5,4.0,18.0,R-5,32.677079,-117.107167,16.0,16.0,23.0,awash ethiopian restaurant,79.0,Food & Drinks
6,2.0,16.0,R-6,32.890137,-117.150878,13.0,3.0,12.0,gueros taco shop,46.0,Only Food
7,4.0,18.0,R-7,32.773069,-117.156144,16.0,16.0,23.0,the fire spot,79.0,Food & Drinks
8,18.0,4.0,R-8,33.016118,-117.075608,2.0,20.0,18.0,up2you cafe llc,62.0,Only Drinks
9,4.0,18.0,R-9,32.708791,-117.160357,16.0,16.0,23.0,cold beers & cheeseburgers,79.0,Food & Drinks


In [9]:
long_max = location_counts['longitude'].max()
long_min = location_counts['longitude'].min()
lat_max = location_counts['latitude'].max()
lat_min = location_counts['latitude'].min()
lat_dif = lat_max - lat_min
long_dif = long_max - long_min
NUMBER_BLOCKS_ROOT = 200 #this means 100 blocks 10x10
lat_gap = lat_dif / NUMBER_BLOCKS_ROOT
long_gap = long_dif / NUMBER_BLOCKS_ROOT

def classify_blocks(s):
    location_counts.loc[location_counts['id'] == s,'row'] =  (location_counts.loc[location_counts['id'] == s,'latitude'] - lat_min) // lat_gap
    location_counts.loc[location_counts['id'] == s,'col'] = (location_counts.loc[location_counts['id'] == s,'longitude'] - long_min) // long_gap

location_counts['id'].apply(classify_blocks)

#Cleaning results from the dataset
location_counts.loc[location_counts['row'] == -0.0, 'row'] = 0.0
location_counts.loc[location_counts['col'] == -0.0, 'col'] = 0.0
location_counts

,am_early,am_peak,id,latitude,longitude,midday,pm_late,pm_peak,title,total_count,type,row,col
0,2.0,16.0,R-0,32.767243,-117.096294,13.0,3.0,12.0,c r e a m,46.0,Only Food,61.0,159.0
1,2.0,16.0,R-1,32.697974,-117.096250,13.0,3.0,12.0,snow cones y raspados,46.0,Only Food,54.0,159.0
2,4.0,18.0,R-2,32.748683,-117.126968,16.0,16.0,23.0,jalapeno taco shop,79.0,Food & Drinks,59.0,155.0
3,2.0,16.0,R-3,32.697974,-117.096250,13.0,3.0,12.0,tacos el campechano inc,46.0,Only Food,54.0,159.0
4,2.0,16.0,R-4,32.718370,-117.157817,13.0,3.0,12.0,up2you cafe llc,46.0,Only Food,56.0,151.0
5,4.0,18.0,R-5,32.677079,-117.107167,16.0,16.0,23.0,awash ethiopian restaurant,79.0,Food & Drinks,52.0,157.0
6,2.0,16.0,R-6,32.890137,-117.150878,13.0,3.0,12.0,gueros taco shop,46.0,Only Food,72.0,152.0
7,4.0,18.0,R-7,32.773069,-117.156144,16.0,16.0,23.0,the fire spot,79.0,Food & Drinks,61.0,151.0
8,18.0,4.0,R-8,33.016118,-117.075608,2.0,20.0,18.0,up2you cafe llc,62.0,Only Drinks,84.0,161.0
9,4.0,18.0,R-9,32.708791,-117.160357,16.0,16.0,23.0,cold beers & cheeseburgers,79.0,Food & Drinks,55.0,151.0


In [10]:
print(location_counts['row'].min())
print(location_counts['row'].max())
print(location_counts['row'].mean())
print(location_counts['row'].std())
print(location_counts['col'].min())
print(location_counts['col'].max())
print(location_counts['col'].mean())
print(location_counts['col'].std())

0.0
200.0
59.31933409
7.44344168904
0.0
199.0
151.35035944
5.87655449383


In [36]:
total_loc_counts = location_counts[['pm_peak','row','col']]
total_loc_counts

,pm_peak,row,col
0,12.0,61.0,159.0
1,12.0,54.0,159.0
2,23.0,59.0,155.0
3,12.0,54.0,159.0
4,12.0,56.0,151.0
5,23.0,52.0,157.0
6,12.0,72.0,152.0
7,23.0,61.0,151.0
8,18.0,84.0,161.0
9,23.0,55.0,151.0


In [37]:
grid = []
for i in range(NUMBER_BLOCKS_ROOT):
    for j in range(NUMBER_BLOCKS_ROOT):
        grid.append([i, j, (lat_min+ i*lat_gap), (long_min + j*long_gap)])    
grid = pd.DataFrame(grid)
grid.columns = ['row','col','lat','lon']
grid
#grid.loc[(grid['row']==142) & (grid['col']==378)]

,row,col,lat,lon
0,0,0,32.100000,-118.426461
1,0,1,32.100000,-118.418097
2,0,2,32.100000,-118.409733
3,0,3,32.100000,-118.401368
4,0,4,32.100000,-118.393004
5,0,5,32.100000,-118.384640
6,0,6,32.100000,-118.376276
7,0,7,32.100000,-118.367911
8,0,8,32.100000,-118.359547
9,0,9,32.100000,-118.351183


In [38]:
weighted_grid = total_loc_counts.groupby(['row','col']).sum().sort_values(by='pm_peak',ascending=False)
weighted_grid

pm_peak
row  col               
56.0 151.0  8242.107958
     150.0  2875.473521
59.0 151.0  2776.644507
57.0 150.0  1815.250986
55.0 151.0  1807.638099
56.0 152.0  1394.661479
55.0 152.0  1190.906127
57.0 151.0  1073.880423
59.0 140.0   896.000000
     152.0   858.727958
63.0 140.0   810.000000
68.0 137.0   798.000000
60.0 153.0   770.709085
58.0 151.0   761.811690
66.0 151.0   696.000000
     152.0   687.000000
84.0 161.0   677.000000
59.0 145.0   670.000000
61.0 151.0   616.000000
60.0 154.0   599.609718
70.0 145.0   567.000000
63.0 158.0   563.000000
59.0 155.0   553.000000
65.0 152.0   523.000000
59.0 150.0   443.781690
67.0 152.0   440.000000
64.0 141.0   400.000000
59.0 154.0   398.000000
     147.0   394.000000
74.0 153.0   393.000000
...                 ...
72.0 153.0    12.000000
73.0 144.0    12.000000
     148.0    12.000000
     157.0    12.000000
74.0 141.0    12.000000
     143.0    12.000000
66.0 150.0    12.000000
65.0 162.0    12.000000
60.0 142.0    12.000000
65.0 158.0    12.000000
60.0 164.0    12.000000
61.0 148.0    12.000000
     158.0    12.000000
     159.0    12.000000
     166.0    12.000000
62.0 142.0    12.000000
     149.0    12.000000
     151.0    12.000000
40.0 164.0    12.000000
63.0 149.0    12.000000
     152.0    12.000000
     155.0    12.000000
     163.0    12.000000
     168.0    12.000000
64.0 145.0    12.000000
     178.0    12.000000
65.0 155.0    12.000000
0.0  170.0    11.616620
82.0 170.0     4.568592
54.0 152.0     1.159366

[501 rows x 1 columns]

In [39]:
# TODO:: Figure out how to merge these two dataframes where the values in grid 
#weighted_grid = pd.merge(weighted_grid, grid, on=['row')
big_grid = pd.merge(grid, 
                weighted_grid.reset_index(), 
                left_on=['row','col'], 
                right_on=['row','col'], 
                how='right')
big_grid['pm_peak']

0       11.61662
1       12.00000
2      208.00000
3      182.00000
4       35.00000
5       77.00000
6      193.00000
7       78.00000
8       65.00000
9       70.00000
10      12.00000
11      24.00000
12      42.00000
13      64.00000
14      12.00000
15      18.00000
16      24.00000
17      12.00000
18     160.00000
19      83.00000
20      12.00000
21     170.00000
22      47.00000
23      30.00000
24      35.00000
25      12.00000
26      18.00000
27      12.00000
28      12.00000
29     100.00000
         ...    
471     12.00000
472     58.00000
473    677.00000
474     53.00000
475     23.00000
476     12.00000
477     18.00000
478     12.00000
479     23.00000
480     12.00000
481     12.00000
482     12.00000
483     12.00000
484     12.00000
485     12.00000
486     23.00000
487     12.00000
488     12.00000
489     12.00000
490     12.00000
491     12.00000
492     12.00000
493     12.00000
494     12.00000
495     18.00000
496     18.00000
497     24.00000
498     23.000

In [42]:
mean = big_grid['pm_peak'].mean()
std = big_grid['pm_peak'].std()
i = grid.loc[big_grid['pm_peak'].max(), 'pm_peak']
mean, std, i

TypeError: cannot do label indexing on <class 'pandas.indexes.range.RangeIndex'> with these indexers [8242.10795775] of <type 'numpy.float64'>

In [41]:
big_grid.to_csv('PM_Peak_Heat_Map.csv')